In [ ]:
# In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity 
# differ between months. To achieve this, follow these steps:

import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine, text
import getpass

# Ask for MySQL password securely
password = getpass.getpass("Enter your MySQL password: ")

In [ ]:
# 1. Establish a connection between Python and the Sakila database.

db = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+db
engine = create_engine(connection_string)

with engine.connect() as connection:
    query = text("SHOW TABLES;")
    result = connection.execute(query)
    tables = pd.DataFrame(result.fetchall(), columns=result.keys())

print(tables)

              Tables_in_sakila
0                        actor
1                   actor_info
2                      address
3                     category
4                         city
5                      country
6                     customer
7                customer_list
8                         film
9                   film_actor
10               film_category
11                   film_list
12                   film_text
13                   inventory
14                    language
15  nicer_but_slower_film_list
16                     payment
17                      rental
18          rental_information
19      sales_by_film_category
20              sales_by_store
21                       staff
22                  staff_list
23                       store


In [19]:
# 2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from 
# the Sakila database as a Pandas DataFrame. The function should take in three parameters:

# engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
# month: an integer representing the month for which rental data is to be retrieved.
# year: an integer representing the year for which rental data is to be retrieved.

# The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila 
# database, and return it as a pandas DataFrame.

def rentals_month(engine, month, year):
    with engine.connect() as connection:
        query = text("""
            SELECT *
            FROM rental
            WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year;
        """)
        result = connection.execute(query, {"month": month, "year": year})
        rentals_df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return rentals_df

In [22]:
# 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month 
# and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

# The function should also include the month and year as parameters and use them to name the new column according to the month and 
# year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

# Hint: Consider making use of pandas groupby()

def rental_count_month(rentals_df, month, year):
    column_name = f"rentals_{month:02d}_{year}"
    rental_counts = rentals_df.groupby("customer_id").size().reset_index(name=column_name)
    return rental_counts

In [23]:
# 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each 
# customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the 
# difference between the number of rentals in the two months.

def compare_rentals(df1, df2):
    combined = pd.merge(df1, df2, on="customer_id", suffixes=("_may", "_jun"))
    combined["difference"] = combined["rentals_05_2005"] - combined["rentals_06_2005"]
    return combined